<a href="https://colab.research.google.com/github/tanmana5/mangrove-diaries.github.io/blob/main/Sunderbans_Mangrove_Mapper_(Updated_to_use_CSV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import folium
from geopy.geocoders import Nominatim
import os
import csv

# --- Constants and Parameters ---
# The name of the geographic area to search for
LOCATION_NAME = "Sunderbans"
# The name of the output HTML file
OUTPUT_FILE = "sunderbans_mangrove_map.html"
# The name of the external data file
DATA_FILE = "mangrove_data.csv"

# --- Example of the mangrove_data.csv file format ---
# You will need to create a file named "mangrove_data.csv" with content like this:
# latitude,longitude,description
# 21.73,88.58,Sajnekhali Wildlife Sanctuary
# 21.78,88.62,Sudhanyakhali Watch Tower
# 21.84,88.94,Central Sunderbans Area
# 21.68,88.80,Bay of Bengal Coast
# 21.65,88.38,Western Delta
# 21.90,88.85,Central Point 2
# 21.79,88.42,Creek Habitat
# 21.71,88.69,Well-known Mangrove
# 22.02,88.80,North-eastern Part
# 22.09,88.75,Further Inland

# --- Functions ---

def get_location_coordinates(location_name):
    """
    Uses geopy to get the latitude and longitude for a given location.

    Args:
        location_name (str): The name of the place.

    Returns:
        tuple: A tuple containing (latitude, longitude) or (None, None) if not found.
    """
    geolocator = Nominatim(user_agent="sunderbans_mangrove_mapper")
    try:
        location = geolocator.geocode(location_name)
        if location:
            return (location.latitude, location.longitude)
        else:
            print(f"Could not find coordinates for {location_name}.")
            return (None, None)
    except Exception as e:
        print(f"An error occurred: {e}")
        return (None, None)

def generate_map(center_coords):
    """
    Creates a folium map centered at the given coordinates.

    Args:
        center_coords (tuple): A tuple (latitude, longitude) for the map center.

    Returns:
        folium.Map: The generated map object.
    """
    # Create the map, centered on the Sunderbans
    m = folium.Map(location=center_coords, zoom_start=10, tiles="OpenStreetMap")
    return m

def load_mangrove_data(filename):
    """
    Loads mangrove data (latitude, longitude, description) from a CSV file.

    Args:
        filename (str): The path to the CSV file.

    Returns:
        list: A list of tuples, each containing (latitude, longitude, description),
              or an empty list if the file is not found or an error occurs.
    """
    mangrove_data = []
    try:
        with open(filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            # Skip the header row
            next(reader)
            for row in reader:
                try:
                    lat, lon, desc = row
                    mangrove_data.append((float(lat), float(lon), desc))
                except (ValueError, IndexError) as e:
                    print(f"Skipping malformed row: {row}. Error: {e}")
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        print("Please create this file and add mangrove data.")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

    return mangrove_data


def add_mangrove_locations(map_obj, locations):
    """
    Adds mangrove locations to the map from a pre-defined list.

    Args:
        map_obj (folium.Map): The map object to add markers to.
        locations (list): A list of tuples, each containing (lat, lon, description).
    """
    # A feature group to organize the mangrove markers
    mangrove_group = folium.FeatureGroup(name="Mangrove Locations")

    # Iterate through the coordinates and add a marker for each
    for lat, lon, desc in locations:
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color="green", icon="leaf", prefix="fa"),
            tooltip=desc
        ).add_to(mangrove_group)

    # Add the group to the map
    mangrove_group.add_to(map_obj)

    # Add a layer control so the user can toggle the mangrove markers
    folium.LayerControl().add_to(map_obj)


def main():
    """
    Main function to orchestrate the mapping process.
    """
    print(f"Generating a map of {LOCATION_NAME} using data from '{DATA_FILE}'...")

    # Get coordinates for the Sunderbans to center the map
    sunderbans_coords = get_location_coordinates(LOCATION_NAME)

    # Load mangrove data from the CSV file
    mangrove_locations = load_mangrove_data(DATA_FILE)

    if sunderbans_coords[0] is not None and mangrove_locations:
        # Create the map
        sunderbans_map = generate_map(sunderbans_coords)

        # Add the mangrove locations to the map
        add_mangrove_locations(sunderbans_map, mangrove_locations)

        # Save the map to an HTML file
        sunderbans_map.save(OUTPUT_FILE)

        # Provide feedback to the user
        print(f"Map saved successfully as '{OUTPUT_FILE}' in your current directory.")
        print(f"Open this file in a web browser to view the interactive map.")
    else:
        print("Map generation failed. Check for location errors or ensure your CSV file is not empty.")


if __name__ == "__main__":
    main()

Generating a map of Sunderbans using data from 'mangrove_data.csv'...
Error: The file 'mangrove_data.csv' was not found.
Please create this file and add mangrove data.
Map generation failed. Check for location errors or ensure your CSV file is not empty.
